In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Supress warnings
import warnings
warnings.filterwarnings('ignore')

# Scikit Learn
import sklearn
sklearn.set_config(display='diagram')
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet  
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.feature_selection import SelectKBest, f_regression

sns.set_style()
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 9)

from IPython import get_ipython
ipython = get_ipython()

if 'ipython' in globals():
    ipython.magic('matplotlib inline')

In [24]:
df = pd.read_csv('data/combined.csv', index_col = 't')

In [25]:
df.head()

,market_mvrv_more_155,addresses_accumulation_balance,addresses_new_non_zero_count,indicators_sopr_account_based,indicators_sopr_adjusted,transactions_rate,market_mvrv,addresses_accumulation_count,entities_receiving_count,market_marketcap_usd,indicators_nvts,addresses_min_1k_count,indicators_puell_multiple,transactions_transfers_volume_entity_adjusted_sum,entities_net_growth_count,indicators_sopr_more_155,market_price_drawdown_relative,entities_min_1k_count,market_price_realized_usd,market_mvrv_less_155,indicators_net_unrealized_profit_loss_account_based,transactions_transfers_volume_adjusted_sum,indicators_hodled_lost_coins,indicators_sopr,addresses_profit_relative,entities_new_count,market_mvrv_z_score,indicators_mvrv_account_based,transactions_size_sum,entities_profit_relative,transactions_transfers_volume_miners_net,indicators_nupl_more_155_account_based,indicators_liveliness,indicators_liveliness_account_based,transactions_entity_adjusted_count,transactions_count,addresses_min_10k_count,indicators_reserve_risk,indicators_realized_profit,addresses_count,indicators_nupl_more_155,addresses_active_count,market_marketcap_realized_usd,market_price_usd_close,addresses_non_zero_count,indicators_nvt,entities_active_count,indicators_net_unrealized_profit_loss,indicators_sopr_less_155
t,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-01,NaN,70018.15,134.0,NaN,NaN,0.000023,NaN,8.0,2.0,NaN,NaN,9.0,NaN,6700.00,48.0,NaN,NaN,92.0,0.04951,NaN,NaN,100.00,1.616534e+06,NaN,NaN,50.0,NaN,NaN,316.0,NaN,6500.0,NaN,0.008960,0.009140,134.0,2.0,4.0,NaN,NaN,32745.0,NaN,4.0,80758.2365,NaN,29959.0,NaN,4.0,NaN,NaN
2010-01-02,NaN,70018.15,126.0,NaN,NaN,0.000000,NaN,8.0,0.0,NaN,NaN,9.0,NaN,6300.00,37.0,NaN,NaN,96.0,0.04951,NaN,NaN,0.00,1.622842e+06,NaN,NaN,37.0,NaN,NaN,NaN,NaN,6300.0,NaN,0.008891,0.009069,126.0,0.0,4.0,NaN,NaN,32871.0,NaN,0.0,81067.6740,NaN,30085.0,NaN,0.0,NaN,NaN
2010-01-03,NaN,70018.15,186.0,NaN,NaN,0.000000,NaN,8.0,0.0,NaN,NaN,9.0,NaN,9300.00,61.0,NaN,NaN,97.0,0.04951,NaN,NaN,0.00,1.632199e+06,NaN,NaN,61.0,NaN,NaN,NaN,NaN,9300.0,NaN,0.008806,0.008983,186.0,0.0,4.0,NaN,NaN,33057.0,NaN,0.0,81528.1170,NaN,30271.0,NaN,0.0,NaN,NaN
2010-01-04,NaN,90722.50,183.0,NaN,NaN,0.000023,NaN,9.0,3.0,NaN,NaN,9.0,NaN,29854.35,57.0,NaN,NaN,98.0,0.04951,NaN,NaN,20804.35,1.639507e+06,NaN,NaN,59.0,NaN,NaN,9649.0,NaN,5050.0,NaN,0.009810,0.009984,183.0,2.0,4.0,NaN,NaN,33240.0,NaN,85.0,81976.1825,NaN,30372.0,NaN,5.0,NaN,NaN
2010-01-05,NaN,90722.50,198.0,NaN,NaN,0.000000,NaN,9.0,0.0,NaN,NaN,9.0,NaN,9900.00,62.0,NaN,NaN,98.0,0.04951,NaN,NaN,0.00,1.647175e+06,NaN,NaN,62.0,NaN,NaN,NaN,NaN,9900.0,NaN,0.011092,0.011264,198.0,0.0,4.0,NaN,NaN,33438.0,NaN,0.0,82466.3315,NaN,30570.0,NaN,0.0,NaN,NaN


In [42]:
((df.isnull().sum())/len(df)*100).sort_values(ascending = False)

market_mvrv_more_155                                   0.0
addresses_accumulation_balance                         0.0
addresses_new_non_zero_count                           0.0
indicators_sopr_account_based                          0.0
indicators_sopr_adjusted                               0.0
transactions_rate                                      0.0
market_mvrv                                            0.0
addresses_accumulation_count                           0.0
entities_receiving_count                               0.0
market_marketcap_usd                                   0.0
indicators_nvts                                        0.0
addresses_min_1k_count                                 0.0
indicators_puell_multiple                              0.0
transactions_transfers_volume_entity_adjusted_sum      0.0
entities_net_growth_count                              0.0
indicators_sopr_more_155                               0.0
market_price_drawdown_relative                         0

In [27]:
# df[df['indicators_sopr_more_155'].isnull() == False]

,market_mvrv_more_155,addresses_accumulation_balance,addresses_new_non_zero_count,indicators_sopr_account_based,indicators_sopr_adjusted,transactions_rate,market_mvrv,addresses_accumulation_count,entities_receiving_count,market_marketcap_usd,indicators_nvts,addresses_min_1k_count,indicators_puell_multiple,transactions_transfers_volume_entity_adjusted_sum,entities_net_growth_count,indicators_sopr_more_155,market_price_drawdown_relative,entities_min_1k_count,market_price_realized_usd,market_mvrv_less_155,indicators_net_unrealized_profit_loss_account_based,transactions_transfers_volume_adjusted_sum,indicators_hodled_lost_coins,indicators_sopr,addresses_profit_relative,entities_new_count,market_mvrv_z_score,indicators_mvrv_account_based,transactions_size_sum,entities_profit_relative,transactions_transfers_volume_miners_net,indicators_nupl_more_155_account_based,indicators_liveliness,indicators_liveliness_account_based,transactions_entity_adjusted_count,transactions_count,addresses_min_10k_count,indicators_reserve_risk,indicators_realized_profit,addresses_count,indicators_nupl_more_155,addresses_active_count,market_marketcap_realized_usd,market_price_usd_close,addresses_non_zero_count,indicators_nvt,entities_active_count,indicators_net_unrealized_profit_loss,indicators_sopr_less_155
t,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-12-23,3.231542,4.585587e+05,681.0,1.184698,NaN,0.004329,2.290016,882.0,257.0,1.233581e+06,69.476566,346.0,NaN,17729.700000,114.0,2.744538,-0.492342,522.0,0.108694,1.394285,0.556128,62713.540000,4.057768e+06,1.025355,0.939098,211.0,2.065642,2.257837,127731.0,0.945200,5061.980000,0.799337,0.181225,0.182175,506.0,374.0,57.0,0.014541,3.974164e+02,170925.0,0.690550,784.0,5.386780e+05,0.248753,64406.0,79.024402,324.0,0.559091,1.166872
2010-12-24,3.316539,4.608220e+05,615.0,0.967090,NaN,0.004433,2.294736,904.0,236.0,1.240200e+06,69.275454,347.0,NaN,19783.927481,69.0,2.744538,-0.493878,525.0,0.108890,1.385889,0.553989,60406.229187,4.065808e+06,0.993452,0.934750,160.0,2.068072,2.259420,116313.0,0.942328,4552.385992,0.798560,0.180826,0.181787,517.0,383.0,57.0,0.014409,1.050780e+02,171540.0,0.698481,707.0,5.404544e+05,0.248000,64589.0,82.165367,304.0,0.556993,0.968438
2010-12-25,3.447517,4.596881e+05,819.0,1.100668,NaN,0.005394,2.283546,922.0,248.0,1.240606e+06,68.168427,348.0,NaN,34114.017431,96.0,2.744538,-0.490000,526.0,0.109275,1.388628,0.556029,112597.040000,4.073744e+06,1.022911,0.937665,184.0,2.047710,2.248354,131257.0,0.945610,3204.827431,0.799899,0.180613,0.181586,624.0,466.0,58.0,0.014203,4.285243e+02,172359.0,0.709936,915.0,5.432803e+05,0.249900,64882.0,44.154802,316.0,0.558914,1.068515
2010-12-26,4.069384,4.581898e+05,720.0,1.151690,NaN,0.005081,2.337016,945.0,260.0,1.275171e+06,69.395511,350.0,NaN,24241.309038,79.0,2.744538,-0.459184,528.0,0.109589,1.455695,0.580054,70038.780164,4.081981e+06,1.032988,0.968602,169.0,2.129715,2.300469,134987.0,0.967997,2865.188874,0.811287,0.180152,0.181133,562.0,439.0,58.0,0.014386,4.337047e+02,173079.0,0.754263,885.0,5.456406e+05,0.265000,65091.0,71.088474,326.0,0.582816,1.111867
2010-12-27,4.131722,4.620030e+05,607.0,1.004255,NaN,0.004167,2.334431,953.0,254.0,1.278616e+06,68.922773,353.0,NaN,56188.234962,88.0,2.744538,-0.459184,523.0,0.109824,1.449247,0.579092,81517.658317,4.087583e+06,1.012033,0.967560,187.0,2.119549,2.313249,147845.0,0.967178,-9186.098555,0.811155,0.180393,0.181383,500.0,360.0,59.0,0.014215,7.554248e+02,173686.0,0.757970,985.0,5.477206e+05,0.265000,65003.0,61.179996,325.0,0.581680,1.006392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-01,1.972435,2.339565e+06,390458.0,1.014785,1.029550,3.658067,1.605087,456787.0,179362.0,1.655956e+11,32.532021,2146.0,1.173380,304698.356350,7137.0,1.408601,-0.532584,1731.0,5723.786509,0.952407,0.366031,690371.906605,7.021816e+06,1.012579,0.766710,10267

In [28]:
df.dropna(inplace = True)
df.shape

(3035, 49)

In [48]:
market_cols = [col for col in df.columns if 'market' in col]
market_cols

['market_mvrv_more_155',
 'market_mvrv',
 'market_marketcap_usd',
 'market_price_drawdown_relative',
 'market_price_realized_usd',
 'market_mvrv_less_155',
 'market_mvrv_z_score',
 'market_marketcap_realized_usd',
 'market_price_usd_close']

In [58]:
indicator_cols = [col for col in df.columns if 'indicators' in col]
indicator_cols

['indicators_sopr_account_based',
 'indicators_sopr_adjusted',
 'indicators_nvts',
 'indicators_puell_multiple',
 'indicators_sopr_more_155',
 'indicators_net_unrealized_profit_loss_account_based',
 'indicators_hodled_lost_coins',
 'indicators_sopr',
 'indicators_mvrv_account_based',
 'indicators_nupl_more_155_account_based',
 'indicators_liveliness',
 'indicators_liveliness_account_based',
 'indicators_reserve_risk',
 'indicators_realized_profit',
 'indicators_nupl_more_155',
 'indicators_nvt',
 'indicators_net_unrealized_profit_loss',
 'indicators_sopr_less_155']

In [78]:
X = df.drop(columns = market_cols + indicator_cols)
y = df['market_price_realized_usd']

In [79]:
X.columns

Index(['addresses_accumulation_balance', 'addresses_new_non_zero_count',
       'transactions_rate', 'addresses_accumulation_count',
       'entities_receiving_count', 'addresses_min_1k_count',
       'transactions_transfers_volume_entity_adjusted_sum',
       'entities_net_growth_count', 'entities_min_1k_count',
       'transactions_transfers_volume_adjusted_sum',
       'addresses_profit_relative', 'entities_new_count',
       'transactions_size_sum', 'entities_profit_relative',
       'transactions_transfers_volume_miners_net',
       'transactions_entity_adjusted_count', 'transactions_count',
       'addresses_min_10k_count', 'addresses_count', 'addresses_active_count',
       'addresses_non_zero_count', 'entities_active_count'],
      dtype='object')

In [80]:
X.shape

(3035, 22)

In [81]:
y.shape

(3035,)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [83]:
scaler = StandardScaler()

In [84]:
lr = LinearRegression()

In [85]:
lr.fit(X_train, y_train)

LinearRegression()

In [86]:
lr.score(X_train, y_train)

0.986703411537047

In [87]:
lr.score(X_test, y_test)

0.9860311443816832

In [ ]:
# Time-Series
# SARIMA, SARIMAX // ARIMA, ARIMAX

# Recurrent Neural Net

# Bayesian Model (facebook profit library)

In [ ]:
# sktime